<a href="https://colab.research.google.com/github/WhiteAndBlackFox/nlp/blob/BERT/BERT%26GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Модель BERT и GPT

## Доставляем библиотеки и импортируем их

In [43]:
!pip install transformers transformers[sentencepiece] sentencepiece datasets pymorphy2 stop-words annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [44]:
import numpy as np
import pandas as pd
import collections
import string
import nltk
from pymorphy2 import MorphAnalyzer

import tensorflow as tf

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from datasets import load_dataset, load_metric

from annoy import AnnoyIndex

from stop_words import get_stop_words

from gensim.models import Word2Vec

from tqdm import tqdm; tqdm.pandas()

## Дополнительные функции

In [45]:
def paraphrase(text):
    """
      Функция парафраза
    """
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    out = model.generate(**x, encoder_no_repeat_ngram_size=5, num_beams=6, max_length=max_size)
    return tokenizer.decode(out[0], skip_special_tokens=True)


def preprocess_txt(txt):
  """
    Функция подготовки данных
  """
  txt = "".join(i for i in txt.strip() if i not in exclude).split()
  txt = [morpher.parse(i.lower())[0].normal_form for i in txt]
  txt = [i for i in txt if i not in sw and i != ""]
  return [i for i in txt if len(i) > 2]

def prepare(question, index, model, index_map, count_answer=3):
  """
    Предстказание ответа
  """
  question = preprocess_txt(question)
  vector = np.zeros(300)
  norm = 0
  for word in question:
      if word in model.wv:
          vector += model.wv[word]
          norm += 1
  if norm > 0:
      vector = vector / norm
  answers = index.get_nns_by_vector(vector, count_answer)
  return [index_map[i] for i in answers]

## Глобальные переменные

In [47]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
w2v_index = AnnoyIndex(300 ,'angular')

## 1. Решим задачу парафразы на датасете - https://huggingface.co/datasets/merionum/ru_paraphraser

In [4]:
dataset = load_dataset("merionum/ru_paraphraser")
dataset

Using custom data configuration merionum--ru_paraphraser-1a7592429d7be082


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-1a7592429d7be082/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 7227
    })
    test: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 1924
    })
})

In [5]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rut5-base-paraphraser")
model = AutoModelForSeq2SeqLM.from_pretrained("cointegrated/rut5-base-paraphraser")
model

Downloading:   0%|          | 0.00/315 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/808k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


Downloading:   0%|          | 0.00/724 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/932M [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(30000, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(30000, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [6]:
for i in range(10):
  text = dataset["train"][i]['text_1']
  print(f"Исходный текст: {text}\nПарафраз:{paraphrase(text)}\n", "-" * 100)

Исходный текст: Полицейским разрешат стрелять на поражение по гражданам с травматикой.
Парафраз:Полицейские разрешат стрелять в нападение на граждан с травматизмом.
 ----------------------------------------------------------------------------------------------------
Исходный текст: Право полицейских на проникновение в жилище решили ограничить.
Парафраз:Полицейские решили ограничить право проникновения в жилище.
 ----------------------------------------------------------------------------------------------------
Исходный текст: Президент Египта ввел чрезвычайное положение в мятежных городах.
Парафраз:Президент Египта объявил чрезвычайное положение в городах-мятежниках.
 ----------------------------------------------------------------------------------------------------
Исходный текст: Вернувшихся из Сирии россиян волнует вопрос трудоустройства на родине.
Парафраз:Россияне, вернувшиеся из Сирии, волнуют вопрос трудоустройства в своей родине.
 ---------------------------------------------

## 2. Попробуем обучить вопронос-ответную систему

In [22]:
dataset = load_dataset('blinoff/medical_qa_ru_data')
dataset

  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['date', 'categ', 'theme', 'desc', 'ans', 'spec10'],
        num_rows: 190335
    })
})

In [23]:
dataset["train"]

Dataset({
    features: ['date', 'categ', 'theme', 'desc', 'ans', 'spec10'],
    num_rows: 190335
})

In [35]:
df_train = pd.DataFrame(dataset["train"])
df_train.head()

,date,categ,theme,desc,ans,spec10
0,"8 Октября 2017, 11:55",Оториноларингология,Применение Ларипронта.,"Ларипронт 20 талеток,через каждые 2-3 часа.Оче...",Что вы им лечите? Длительность приема Ларипрон...,Отоларинголог
1,"20 Февраля 2019, 13:24",Акушерство,Беременность,"Здравствуйте, я на 7-8 неделе беременности. С ...","Здравствуйте, это может быть признаком раннего...",
2,"17 Марта 2015, 18:31",Другое,гинекология,Здравствуйте месячные должны придти 23 марта в...,Выполните исследование хгч,
3,"13 Января 2019, 19:38",Терапия,Занятия спорта после сдачи крови,"Завтра иду с утра сдавать кровь ТТГ, Т4СВ, Кал...","Можно.;\nЗдравствуйте , да, попейте сладкого ч...",Терапевт
4,"28 Ноября 2017, 21:58",Другое,Таблетки,Мне прописали пить Аллохол. Врач написала пить...,Препарат принимается после еды. Уточните это ...,


In [38]:
# Чистим данные
df_train = df_train.dropna().reset_index()
df_train["preprocess_ans"] = df_train["ans"].progress_apply(lambda txt: preprocess_txt(txt))

100%|██████████| 190335/190335 [36:02<00:00, 88.03it/s]


In [53]:
modelW2V = Word2Vec(sentences=df_train["preprocess_ans"], size=300, window=5, min_count=3, workers=8)

In [54]:
index_map = {}
counter = 0

for i in range(len(df_train)):
    n_w2v = 0
    index_map[counter] = df_train['ans'][i]
    question = preprocess_txt(df_train['desc'][i])
        
    vector_w2v = np.zeros(300)
    for word in question:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
          
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        
    w2v_index.add_item(counter, vector_w2v)
            
    counter += 1

w2v_index.build(10)

True

In [55]:
candidate_answer = prepare("Частая мигрень головы. Что делать?", w2v_index, modelW2V, index_map, count_answer=3)
candidate_answer[0]

'Здравствуйте, пульс 106 это симптом только, какое у вас давление??;\n110/67;\nНе испытываете ли Вы чувство тревоги и страха?;\nКакое у вас давление?;\nМожет.'